In [81]:
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, KFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [39]:
base = pd.read_csv('../data/bank-full.csv_v2', sep=',')
base

,age,education,default,balance,housing,loan,day,month,duration,campaign,...,married,single,cellular,telephone,unknown.1,failure,other,success,unknown.2,y
0,58,2,0,2143,1,0,5,5,261,1,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,no
1,44,1,0,29,1,0,5,5,151,1,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,no
2,33,1,0,2,1,1,5,5,76,1,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,no
3,47,3,0,1506,1,0,5,5,92,1,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,no
4,33,3,0,1,0,0,5,5,198,1,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,2,0,825,0,0,17,11,977,3,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,yes
45207,71,0,0,1729,0,0,17,11,456,2,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,yes
45208,72,1,0,5715,0,0,17,11,1127,5,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,yes
45209,57,1,0,668,0,0,17,11,508,4,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,no


In [40]:
X = base.iloc[:,0:34]
y = base.iloc[:,34]

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=0)

In [95]:
rfc = RandomForestClassifier(n_jobs=-1)

In [67]:
n_estimators = [int(i) for i in np.linspace(start=500, stop=1000, num=20)]
max_depth = [int(i) for i in np.linspace(10, 100, num=5)]
max_depth.append(None)
max_features = ['auto', 'sqrt']
min_samples_split = [2, 3, 4]
min_samples_leaf = [2, 3, 4]

parm_space = {'n_estimators':n_estimators,
                'max_depth':max_depth,
                'max_features':max_features,
                'min_samples_split':min_samples_split,
                'min_samples_leaf':min_samples_leaf}


In [94]:
randomSearch = RandomizedSearchCV(estimator=rfc,
                                    param_distributions=parm_space,
                                    n_iter=10,
                                    cv=5,
                                    verbose=3,
                                    random_state=0)
randomSearch.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'max_depth': [10, 32, 55, 77, 100,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [2, 3, 4],
                                        'min_samples_split': [2, 3, 4],
                                        'n_estimators': [500, 526, 552, 578,
                                                         605, 631, 657, 684,
                                                         710, 736, 763, 789,
                                                         815, 842, 868, 894,
                                                         921, 947, 973, 1000]},
                   random_state=0, verbose=3)

In [96]:
randomSearch.best_params_

{'n_estimators': 842,
 'min_samples_split': 3,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 100}

In [97]:
model = RandomForestClassifier(n_estimators=842, 
                                min_samples_split=3, 
                                min_samples_leaf=2,
                                max_features='sqrt', 
                                max_depth=100,
                                n_jobs=-1)

In [84]:
Kfold = KFold(n_splits=10, shuffle=True, random_state=0)
scores = cross_val_score(model, X, y, cv=Kfold)

In [85]:
scores

array([0.9068996 , 0.89736784, 0.9071002 , 0.90068569, 0.90422473,
       0.90134926, 0.90776377, 0.90422473, 0.90798496, 0.90886972])

In [98]:
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=100, min_samples_leaf=2, min_samples_split=3,
                       n_estimators=842, n_jobs=-1)

In [99]:
predict = model.predict(X_test)

In [100]:
evaluation = accuracy_score(y_test, predict)
evaluation

0.900807254229791

In [101]:
confusion = confusion_matrix(y_test, predict)
confusion

array([[7758,  222],
       [ 675,  388]], dtype=int64)

In [102]:
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

          no       0.92      0.97      0.95      7980
         yes       0.64      0.37      0.46      1063

    accuracy                           0.90      9043
   macro avg       0.78      0.67      0.70      9043
weighted avg       0.89      0.90      0.89      9043



In [80]:
#pickle.dump(model, open('../model/model.pkl', 'wb'))